In [1]:
import sys
sys.path.append("..")   # this allows us to import from sibling directory

from code_to_import.dataset_preprocessing import ProductionModeDataset

import torch
from sklearn.inspection import permutation_importance
import numpy as np
import uproot
import torch.nn as nn
from matplotlib import pyplot as plt

from code_to_import.Classifier_module import Classifier
from code_to_import.Classifier_module import Three_Layer_Classifier

from torch.autograd import Variable


In [74]:
################# THIS IS WHERE YOU CHOOSE WHAT TO LOAD ################################

correlation_cut = -1.0   # if negative none, current other options are 0.9 and 0.8, but more can be added

cut_version = -1   # -1 means OG, otherwise can be 2 or 3 (or more if added)

event_type = "emu"  # could be ee, emu, mumu

model_name = "threeLayerModel_" + event_type + "_corrCut_" + str(correlation_cut)

if cut_version > 0:
    model_name += "cutV" + str(args.cut_version)

In [75]:
root_path = "/depot-new/cms/top/mcnama20/TopSpinCorr-Run2-Entanglement/CMSSW_10_2_22/src/TopAnalysis/Configuration/analysis/diLeptonic/three_files/Nominal"

file = root_path + "/" + event_type + "_modified_root_1.root"

path_to_model = "../models/three_layers/" + model_name + ".pt"

In [76]:
# data_object = ProductionModeDataset(file)
# x = data_object.get_eval_data()

x = np.load("../data/three_layers/train_val_datasets/validation_dataset_" + model_name + ".npy")


In [77]:
############ remove parts of dataset not used in nn input and get output goals (target) and weights ####################
l = x.shape[1]   # get the initial length of the validation dataset
weight = x[:,l-2]   # get an array of weights for plotting
target = x[:,l-3]   # get targets
target = Variable(torch.from_numpy(target).type(torch.LongTensor))   # reformate target into a tensor

######################### remove excess inputs from x and call new variable y
y = np.transpose(x)   
y = np.delete(y, [l-3, l-2, l-1], 0)
y = np.transpose(y)

# the final validation input is gotten from turning y into a tensor
input = Variable(torch.from_numpy(y).type(torch.Tensor))

In [78]:
############################## Load the model and run eval so we know to evaluate ##################################
try:
    model = Classifier()
    model.load_state_dict(torch.load(path_to_model))
except:
    model = Three_Layer_Classifier(input_size=input.shape[1])
    model.load_state_dict(torch.load(path_to_model))
model.eval()

Three_Layer_Classifier(
  (model): Sequential(
    (0): Linear(in_features=80, out_features=512, bias=True)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): LeakyReLU(negative_slope=0.2, inplace=True)
    (4): Linear(in_features=256, out_features=3, bias=True)
    (5): LeakyReLU(negative_slope=0.2, inplace=True)
  )
)

In [79]:
model(input)[0,:]

tensor([0.4752, 0.9318, 0.1371], grad_fn=<SliceBackward>)

In [80]:
target[0]

tensor(1)

In [81]:
from skorch import NeuralNetClassifier
from sklearn.inspection import permutation_importance

In [82]:
nn = NeuralNetClassifier(model, max_epochs=0, lr=0.00, batch_size=5000)
nn.initialize()

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=Three_Layer_Classifier(
    (model): Sequential(
      (0): Linear(in_features=80, out_features=512, bias=True)
      (1): LeakyReLU(negative_slope=0.2, inplace=True)
      (2): Linear(in_features=512, out_features=256, bias=True)
      (3): LeakyReLU(negative_slope=0.2, inplace=True)
      (4): Linear(in_features=256, out_features=3, bias=True)
      (5): LeakyReLU(negative_slope=0.2, inplace=True)
    )
  ),
)

In [83]:
events = uproot.open(file)
key = events.keys()[0]
input_variables = events[key].keys()

In [84]:
nn.score(input, target, sample_weight = weight)

0.4628837052273744

In [ ]:
r = permutation_importance(nn, input, target, \
                           n_repeats=30, \
                           random_state=0)

for i in r.importances_mean.argsort()[::-1]:
    if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
        print(f"{input_variables[i]:<8}         "
              f"{r.importances_mean[i]:.3f}"
              f" +/- {r.importances_std[i]:.3f}")


In [14]:
# save for use in further analysis
np.save("./results/permutation_importance" + model_name + ".npy", r.importances_mean)
np.save("./results/permutation" + model_name + ".npy", r)